# PHYS20762 - Project - 

David Phelan  
University of Manchester  
March 2025

This project aims to simulate a driven, damped harmonic oscillator.

In [24]:
# Initilisation
import numpy as np
import matplotlib.pyplot as plt

The below function is an Euler solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [25]:
def Euler_undriven(m, k, b, n, x0, v0, dt):
    # Initialisation
    t = np.linspace(0, n*dt, n+1)
    x = np.zeros(n+1)
    v = np.zeros(n+1)
    a = np.zeros(n+1)
    x[0] = x0
    v[0] = v0
    # Euler method
    for i in range(n):
        a[i] = (-k*x[i] - b*v[i])/m
        v[i+1] = v[i] + (-k*x[i] - b*v[i])/m*dt
        x[i+1] = x[i] + v[i]*dt
    return x, v, a, t